# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 9 2023 9:21AM,254536,10,CLI00445.1,"CLINUVEL, Inc.",Released
1,Jan 9 2023 9:21AM,254536,10,CLI00446.1,"CLINUVEL, Inc.",Released
2,Jan 9 2023 9:21AM,254536,10,CLI00447.1,"CLINUVEL, Inc.",Released
3,Jan 9 2023 9:21AM,254536,10,CLI00448.1,"CLINUVEL, Inc.",Released
4,Jan 9 2023 9:21AM,254536,10,CLI00449.1,"CLINUVEL, Inc.",Released
5,Jan 9 2023 9:18AM,254534,12,U1172,"Uniderm USA, Inc",Released
6,Jan 9 2023 9:18AM,254534,12,U1174,"Uniderm USA, Inc",Released
7,Jan 9 2023 9:17AM,254533,10,Clear-8998803,"ClearSpec, LLC",Released
8,Jan 9 2023 9:17AM,254533,10,Clear-8998804,"ClearSpec, LLC",Released
9,Jan 9 2023 9:17AM,254533,10,Clear-9009928,"ClearSpec, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
15,254528,Released,1
16,254530,Released,2
17,254533,Released,3
18,254534,Released,2
19,254536,Released,5


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254528,NaN,NaN,1.0
254530,NaN,NaN,2.0
254533,NaN,NaN,3.0
254534,NaN,NaN,2.0
254536,NaN,NaN,5.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254504,0.0,0.0,2.0
254505,2.0,2.0,7.0
254511,0.0,0.0,8.0
254512,0.0,0.0,11.0
254516,0.0,0.0,9.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254504,0,0,2
254505,2,2,7
254511,0,0,8
254512,0,0,11
254516,0,0,9


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254504,0,0,2
1,254505,2,2,7
2,254511,0,0,8
3,254512,0,0,11
4,254516,0,0,9


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254504,,,2
1,254505,2,2,7
2,254511,,,8
3,254512,,,11
4,254516,,,9


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 9 2023 9:21AM,254536,10,"CLINUVEL, Inc."
5,Jan 9 2023 9:18AM,254534,12,"Uniderm USA, Inc"
7,Jan 9 2023 9:17AM,254533,10,"ClearSpec, LLC"
10,Jan 9 2023 9:09AM,254527,16,"Emersa Waterbox, LLC"
11,Jan 9 2023 9:03AM,254530,10,Emerginnova
13,Jan 9 2023 9:02AM,254528,10,Eye Pharma Inc
14,Jan 9 2023 9:02AM,254520,10,ISDIN Corporation
70,Jan 9 2023 8:58AM,254526,10,"Snap Medical Industries, LLC"
72,Jan 9 2023 8:58AM,254525,12,"Uniderm USA, Inc"
73,Jan 9 2023 8:55AM,254524,10,"Snap Medical Industries, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 9 2023 9:21AM,254536,10,"CLINUVEL, Inc.",,,5
1,Jan 9 2023 9:18AM,254534,12,"Uniderm USA, Inc",,,2
2,Jan 9 2023 9:17AM,254533,10,"ClearSpec, LLC",,,3
3,Jan 9 2023 9:09AM,254527,16,"Emersa Waterbox, LLC",,,1
4,Jan 9 2023 9:03AM,254530,10,Emerginnova,,,2
5,Jan 9 2023 9:02AM,254528,10,Eye Pharma Inc,,,1
6,Jan 9 2023 9:02AM,254520,10,ISDIN Corporation,,,56
7,Jan 9 2023 8:58AM,254526,10,"Snap Medical Industries, LLC",,,2
8,Jan 9 2023 8:58AM,254525,12,"Uniderm USA, Inc",,,1
9,Jan 9 2023 8:55AM,254524,10,"Snap Medical Industries, LLC",,,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 9 2023 9:21AM,254536,10,"CLINUVEL, Inc.",5,,
1,Jan 9 2023 9:18AM,254534,12,"Uniderm USA, Inc",2,,
2,Jan 9 2023 9:17AM,254533,10,"ClearSpec, LLC",3,,
3,Jan 9 2023 9:09AM,254527,16,"Emersa Waterbox, LLC",1,,
4,Jan 9 2023 9:03AM,254530,10,Emerginnova,2,,
5,Jan 9 2023 9:02AM,254528,10,Eye Pharma Inc,1,,
6,Jan 9 2023 9:02AM,254520,10,ISDIN Corporation,56,,
7,Jan 9 2023 8:58AM,254526,10,"Snap Medical Industries, LLC",2,,
8,Jan 9 2023 8:58AM,254525,12,"Uniderm USA, Inc",1,,
9,Jan 9 2023 8:55AM,254524,10,"Snap Medical Industries, LLC",6,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 9 2023 9:21AM,254536,10,"CLINUVEL, Inc.",5,,
1,Jan 9 2023 9:18AM,254534,12,"Uniderm USA, Inc",2,,
2,Jan 9 2023 9:17AM,254533,10,"ClearSpec, LLC",3,,
3,Jan 9 2023 9:09AM,254527,16,"Emersa Waterbox, LLC",1,,
4,Jan 9 2023 9:03AM,254530,10,Emerginnova,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 9 2023 9:21AM,254536,10,"CLINUVEL, Inc.",5,NaN,NaN
1,Jan 9 2023 9:18AM,254534,12,"Uniderm USA, Inc",2,NaN,NaN
2,Jan 9 2023 9:17AM,254533,10,"ClearSpec, LLC",3,NaN,NaN
3,Jan 9 2023 9:09AM,254527,16,"Emersa Waterbox, LLC",1,NaN,NaN
4,Jan 9 2023 9:03AM,254530,10,Emerginnova,2,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 9 2023 9:21AM,254536,10,"CLINUVEL, Inc.",5,0.0,0.0
1,Jan 9 2023 9:18AM,254534,12,"Uniderm USA, Inc",2,0.0,0.0
2,Jan 9 2023 9:17AM,254533,10,"ClearSpec, LLC",3,0.0,0.0
3,Jan 9 2023 9:09AM,254527,16,"Emersa Waterbox, LLC",1,0.0,0.0
4,Jan 9 2023 9:03AM,254530,10,Emerginnova,2,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2545222,93,2.0,2.0
12,509059,3,0.0,0.0
16,254527,1,0.0,0.0
19,509023,19,0.0,0.0
20,254517,1,0.0,0.0
22,509037,2,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2545222,93,2.0,2.0
1,12,509059,3,0.0,0.0
2,16,254527,1,0.0,0.0
3,19,509023,19,0.0,0.0
4,20,254517,1,0.0,0.0
5,22,509037,2,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,93,2.0,2.0
1,12,3,0.0,0.0
2,16,1,0.0,0.0
3,19,19,0.0,0.0
4,20,1,0.0,0.0
5,22,2,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,93.0
1,12,Released,3.0
2,16,Released,1.0
3,19,Released,19.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,22
Status,,,,,,
Completed,2.0,0.0,0.0,0.0,0.0,0.0
Executing,2.0,0.0,0.0,0.0,0.0,0.0
Released,93.0,3.0,1.0,19.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,22
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0
1,Executing,2.0,0.0,0.0,0.0,0.0,0.0
2,Released,93.0,3.0,1.0,19.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,22
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0
1,Executing,2.0,0.0,0.0,0.0,0.0,0.0
2,Released,93.0,3.0,1.0,19.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()